In [6]:
from scipy.sparse import csc_matrix
import trimesh
import skeletor as sk
from util.mesh import MeshProcessor
#mesh = trimesh.load("/Users/ackermand/Documents/Downloads/410_roi1.obj")
mp = MeshProcessor(path = "https://janelia-cosem-datasets.s3.amazonaws.com/jrc_hela-2/neuroglancer/mesh/mito_seg",lod = 0, min_branch_length = 20)
ms = mp.get_mesh(id=45)
trimesh_mesh = trimesh.Trimesh(ms.current_mesh().vertex_matrix(), ms.current_mesh().face_matrix())

AttributeError: 'Trimesh' object has no attribute 'current_mesh'

In [ ]:
trimesh_mesh.show()

In [ ]:
#mesh = sk.pre.fix_mesh(mesh, remove_disconnected=5, inplace=False)
mesh = sk.pre.contract(trimesh_mesh)

In [ ]:
skel = sk.skeletonize.by_tangent_ball(trimesh_mesh)# 
# skel = sk.skeletonize.by_wavefront(trimesh_mesh, waves=1, step_size=2)
#skel = sk.skeletonize.by_teasar(trimesh_mesh, inv_dist = 100)
skel = sk.post.clean_up(skel)


In [ ]:
import navis
import numpy as np
from navis import graph
n = navis.TreeNeuron(skel, soma=None)
pruned = navis.prune_twigs(n,100)
seg_lengths_before = np.array([graph.segment_length(n, s) for s in n.segments])
seg_lengths_after = np.array([graph.segment_length(pruned, s) for s in pruned.segments])

print(seg_lengths_before,seg_lengths_after)
skel2 = sk.Skeleton(pruned.nodes[['node_id', 'parent_id', 'x', 'y', 'z', 'radius']],mesh=trimesh_mesh)
print(n.n_leafs,pruned.n_leafs)
skel2 = sk.post.clean_up(skel2)
n2 = navis.TreeNeuron(skel2, soma=None)
longest_path = navis.longest_neurite(n2, from_root=False)
navis.plot3d([n2,longest_path,trimesh_mesh])

In [ ]:
skel.show(mesh=True)


In [ ]:
import numpy as np
import time
metric = np.random.random(len(mesh.vertices))
t = time.time()
face_metrics = np.mean(metric[mesh.faces.flatten()].reshape((-1,3)),axis=1)


In [ ]:

import numpy as np
import time
e = np.array(mesh.edges)
t = time.time()
temp = csc_matrix((np.ones(len(e)),(e[:,0],e[:,1])), shape=(len(mesh.vertices),len(mesh.vertices)))
print(time.time()-t)
t = time.time()
for i in range(10):
    temp = temp.dot(temp)
print(type(temp))
print(time.time()-t)
rows,cols = temp.nonzero()
cols[rows==0]

In [ ]:
g = nx.from_edgelist(mesh.edges_unique)

In [ ]:
a = dict(nx.all_pairs_shortest_path_length(g,cutoff=5))

In [ ]:
a[0][0]

In [ ]:
import pyglet
window = pyglet.window.Window()
label = pyglet.text.Label('Hello, world',
                          font_name='Times New Roman',
                          font_size=36,
                          x=window.width//2, y=window.height//2,
                          anchor_x='center', anchor_y='center')

@window.event
def on_draw():
    window.clear()
    label.draw()

pyglet.app.run()

In [ ]:
import numpy as np
a = np.arange(10)
a.clip(4,7)
print(a)

In [ ]:
import pymeshlab
import numpy as np
ms = pymeshlab.MeshSet()
ms.load_new_mesh("/Users/ackermand/Documents/Downloads/410_roi1.obj")
# ms.load_new_mesh("./content/MPI-FAUST/meshes/tr_reg_000.ply")
mesh = ms.current_mesh()
t = time.time()
ms.meshing_repair_non_manifold_edges()
t=time.time()
ms.compute_scalar_by_geodesic_distance_from_given_point_per_vertex(np.array[mesh.vertex_matrix()[0,:]])
print(time.time()-t)

In [ ]:
print(mesh.vertex_matrix()[0,:])
t=time.time()
ms.compute_scalar_by_geodesic_distance_from_given_point_per_vertex(startpoint = mesh.vertex_matrix()[0,:],maxdistance=pymeshlab.Percentage(1))
print(time.time()-t)

In [7]:
from util.mesh import MeshProcessor
from tqdm import tqdm
import numpy as np

mp = MeshProcessor(path = "https://janelia-cosem-datasets.s3.amazonaws.com/jrc_hela-2/neuroglancer/mesh/mito_seg",lod = 0, min_branch_length = 20, )
#mp.process_mesh(id=53994)
#mp.process_mesh(id=84602)
#mp.process_mesh(id=322)
#mp.process_mesh(id=116)
momenta = np.zeros((421,3),dtype=np.float32)
for i in tqdm(range(1,422)):
    momenta[i-1,:] = mp.process_mesh(id=i)




  0%|          | 0/421 [00:00<?, ?it/s]


ValueError: setting an array element with a sequence.

In [5]:
from util.mesh import MeshProcessor
from tqdm import tqdm
import numpy as np
import dask
from dask.distributed import Client, progress
import pandas as pd

client = Client(threads_per_worker=2, n_workers=1)
client.cluster.scale(8)
print(client.dashboard_link)
mp = MeshProcessor(path = "https://janelia-cosem-datasets.s3.amazonaws.com/jrc_mus-liver/neuroglancer/mesh/mito_seg",lod = 0, min_branch_length = 100, use_skeletons=True)
#momenta = np.zeros((421,3),dtype=np.float32)
lazy_results = []
for i in range(1,87077,100): #range(1,422,1):#
    lazy_results.append(mp.process_mesh(id=i))
results = dask.compute(*lazy_results)
df = pd.DataFrame.from_records(results)
temp = df[["principal_inertia_component_0","principal_inertia_component_1","principal_inertia_component_2"]].to_numpy()>=0
print(temp.all())


/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 61570 instead
  f"Port {expected} is already in use.\n"


http://127.0.0.1:61570/status


/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/trimesh/util.py:128: RuntimeWarning: invalid value encountered in reciprocal
  norm[valid] **= -1
/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/trimesh/util.py:128: RuntimeWarning: invalid value encountered in reciprocal
  norm[valid] **= -1
/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/trimesh/util.py:128: RuntimeWarning: invalid value encountered in reciprocal
  norm[valid] **= -1
/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/trimesh/util.py:128: RuntimeWarning: invalid value encountered in reciprocal
  norm[valid] **= -1
/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/trimesh/util.py:128: RuntimeWarning: invalid value encountered in reciprocal
  norm[valid] **= -1
/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/trimesh/util.py:128: RuntimeWarning: invalid val

True


In [2]:
temp = df[["principal_inertia_component_0","principal_inertia_component_1","principal_inertia_component_2"]].to_numpy()>=0
temp.all()

True

In [3]:
print(df.to_string())

      id        volume  surface_area  principal_inertia_component_0  principal_inertia_component_normalized_0  principal_inertia_component_1  principal_inertia_component_normalized_1  principal_inertia_component_2  principal_inertia_component_normalized_2  num_fragments  num_branches  longest_path
0      1  6.089691e+06  2.403698e+05                   1.401700e+10                                  0.045627                   1.448537e+11                                  0.471512                   1.483402e+11                                  0.482861              1             0    579.152499
1      2  5.372239e+05  3.709387e+04                   4.831928e+08                                  0.256861                   5.866144e+08                                  0.311839                   8.113388e+08                                  0.431300              1             0    128.265584
2      3  3.795491e+06  1.730737e+05                   1.564522e+10                                  0.

In [1]:
from util.mesh import MeshProcessor
mp = MeshProcessor(path = "https://janelia-cosem-datasets.s3.amazonaws.com/jrc_hela-2/neuroglancer/mesh/mito_seg",lod = 2, min_branch_length = 100, use_skeletons=True,close_holes=True)
_ = mp.get_mesh(276)
#mp.process_mesh(id=325).compute()

/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import trimesh
import numpy as np
mesh = trimesh.load_mesh("41401.ply")
print(mesh.is_watertight)
print(mesh.volume, mesh.area, mesh.principal_inertia_components)
mesh.apply_transform(mesh.principal_inertia_transform)
scene = trimesh.Scene()
scene.add_geometry(mesh)
axis = trimesh.creation.axis(origin_color=[1., 0, 0])
scene.add_geometry(axis)
scene.show()


ValueError: string is not a file: 4701.ply

In [2]:
temp = df[["principal_inertia_component_0","principal_inertia_component_1","principal_inertia_component_2"]].to_numpy()>=0
temp.all()


False

In [ ]:
from sklearn.decomposition import PCA
row_norms = np.sum(momenta,axis=1)
normalized_momenta = momenta/row_norms[:,np.newaxis]
pca = PCA(n_components=3)
pca.fit(normalized_momenta)
transformed = pca.transform(normalized_momenta)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot()#projection='3d')
ax.scatter(normalized_momenta[:,2],normalized_momenta[:,0])
print(normalized_momenta[280,:],normalized_momenta[44,:],normalized_momenta[200,:])

In [5]:
from sklearn.cluster import KMeans,SpectralClustering
from sklearn.preprocessing import StandardScaler
data = df[["longest_path","num_fragments","principal_inertia_component_normalized_0","principal_inertia_component_normalized_1","principal_inertia_component_normalized_2"]].to_numpy()
#data = df[["longest_path","num_fragments"]].to_numpy()

data = StandardScaler().fit(data).transform(data)
#clustering = KMeans(n_clusters=5, random_state=0).fit(data)
clustering = SpectralClustering(n_clusters=3,
             assign_labels='discretize',
             random_state=0).fit(data)
for label in range(3):
    print([int(df.iloc[i]["id"]) for i, x in enumerate(clustering.labels_==label) if x])

[1, 4, 5, 7, 10, 11, 16, 20, 21, 22, 26, 27, 28, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 47, 48, 49, 53, 55, 56, 58, 59, 60, 61, 64, 65, 66, 68, 69, 72, 73, 74, 76, 78, 80, 81, 82, 83, 85, 87, 88, 89, 90, 92, 93, 94, 95, 96, 97, 98, 99, 104, 107, 108, 109, 110, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 128, 129, 130, 132, 133, 137, 139, 143, 145, 146, 147, 148, 149, 150, 151, 152, 156, 157, 158, 159, 160, 161, 162, 164, 166, 168, 169, 170, 172, 173, 175, 179, 180, 181, 182, 184, 186, 187, 189, 190, 191, 192, 193, 197, 199, 200, 201, 205, 206, 207, 208, 209, 210, 211, 213, 214, 215, 216, 217, 218, 221, 222, 224, 225, 226, 228, 229, 230, 231, 233, 235, 236, 237, 239, 240, 242, 244, 247, 249, 250, 251, 253, 255, 256, 258, 259, 260, 263, 266, 268, 270, 271, 272, 273, 276, 277, 278, 280, 281, 284, 285, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 299, 300, 301, 304, 305, 307, 309, 310, 311, 312, 314, 315, 316, 318, 319, 321, 322, 323, 324, 325, 327, 328, 3

In [19]:
int(df.iloc[0]["id"])

1